## PEtab Select FAMOS
In this notebook we'll run the model selection example from `PEtab Select: specification standard and supporting software for automated model selection` by Pathirana et. al with basico. This assumes you installed basico with optional PEtab support using 

    pip install copasi-basico[petab]

We begin by downloading the model:

In [3]:
!wget -q https://github.com/copasi/basico/raw/a4ebfa0575b3a0c569ea7274bed71657207c883e/tests/famos.zip
!unzip -qq -o famos.zip
!rm -f famos.zip

now import basico and petab_select to evaluate the problem. The `evaluate_problem` function will calibrate all models suggested by petab_select returning the best overall found model.

In [4]:
from petab_select import Problem
from basico.petab import evaluate_problem
import basico

the main parameter to tweak is the evaluation function, that one defines what optimization methods should be run on each suggested model. The default evaluation will run a couple of steps of a global optimization method, followed by some steps of a local optimizer. That is of coure rather arbitrary and will not yield the best results in many cases. So lets define an evaluation function here just to see how it is done: 

In [1]:
def eval_fun():
    """Evaluation function for currently loaded model

    This function does take the currently loaded model `get_current_model()` and
    perform parameter estimation on it. 

    It is supposed to return the solution as returned by `run_parameter_estimation`.

    :return: Solution of parameter estimation
    :rtype: pandas.DataFrame
    """

    # run genetic algorithm, important that update_model=True to ensure 
    # that the following parameter estimation is based on the result of the genetic algorithm
    basico.run_parameter_estimation(method=basico.PE.GENETIC_ALGORITHM_SR, update_model=True,
                                    settings={'method': {
                                        'Number of Generations': 5,
                                        'Population Size': 10,
                                        'Stop after # Stalled Generations': 30
                                        }})
    # refine the result with a local optimization method
    sol = basico.run_parameter_estimation(method=basico.PE.NL2SOL, update_model=True,
                                            settings={'method': {
                                                'Iteration Limit': 2000,
                                            }}
                                          )
    return sol


that being done, lets load the petab select problem and evaluate it, depending on the function chosen above, this might take a while. A parallel version is in the works for later

In [5]:
problem = Problem.from_yaml('./famos/petab_select_problem.yaml')
best = evaluate_problem(problem, evaluation=eval_fun, temp_dir='./tmp_selection', delete_temp_files=False)

so lets see what we got, for this we define a print function: 

In [2]:
def print_model(model) -> None:
    """Helper method to view model attributes."""
    print(
        f"""\
Model subspace ID: {model.model_subspace_id}
PEtab YAML location: {model.petab_yaml}
Custom model parameters: {model.parameters}
Model hash: {model.get_hash()}
Model ID: {model.model_id}
{problem.criterion}: {model.get_criterion(problem.criterion, compute=False)}
"""
    )

print_model(best)

NameError: name 'best' is not defined

In [10]:
best.parameters


{'a_0ac_k05': 1,
 'a_0ac_k08': 'estimate',
 'a_0ac_k12': 1,
 'a_0ac_k16': 1,
 'a_k05_k05k08': 1,
 'a_k05_k05k12': 'estimate',
 'a_k05_k05k16': 1,
 'a_k08_k05k08': 1,
 'a_k08_k08k12': 1,
 'a_k08_k08k16': 1,
 'a_k12_k05k12': 'estimate',
 'a_k12_k08k12': 1,
 'a_k12_k12k16': 1,
 'a_k16_k05k16': 1,
 'a_k16_k08k16': 1,
 'a_k16_k12k16': 'estimate',
 'a_k05k08_k05k08k12': 1,
 'a_k05k08_k05k08k16': 1,
 'a_k05k12_k05k08k12': 'estimate',
 'a_k05k12_k05k12k16': 1,
 'a_k05k16_k05k08k16': 1,
 'a_k05k16_k05k12k16': 1,
 'a_k08k12_k05k08k12': 1,
 'a_k08k12_k08k12k16': 1,
 'a_k08k16_k05k08k16': 1,
 'a_k08k16_k08k12k16': 1,
 'a_k12k16_k05k12k16': 1,
 'a_k12k16_k08k12k16': 'estimate',
 'a_k05k08k12_4ac': 1,
 'a_k05k08k16_4ac': 1,
 'a_k05k12k16_4ac': 1,
 'a_k08k12k16_4ac': 'estimate'}

In [11]:
best.estimated_parameters

{'a_0ac_k08': 0.4085094414628759,
 'a_k05_k05k12': 30.888199302449056,
 'a_k12_k05k12': 8.267793085364918,
 'a_k16_k12k16': 10.424806975437063,
 'a_k05k12_k05k08k12': 4.872782833429837,
 'a_k12k16_k08k12k16': 33.03771578079315,
 'a_k08k12k16_4ac': 53.80176047873337,
 'a_b': 0.06675795470746987}